In [1]:
import data_loader

import torch
import torch.nn as nn

import numpy as np

import random
import normalizer
from PIL import Image, ImageDraw, ImageFont

import cv2

import models
import modelutils

In [2]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))




train = data_loader.load("I:/NSU/CV/tests/torch/data/train/coords",
                         "I:/NSU/CV/tests/torch/data/train/images", 
                        firstn = 5000, batchSize = 32, shuffle = True, 
                        displace = True, size = 480, show = False)



scaler = normalizer.MinMaxNormalizer()
scaler.fit([y for _, y in train])

print("Number of batches:", len(train))
for x, y in train:
   print(x.shape, y.shape)
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("devise is: ", device)



condcords08718_3d.txt dataimg08718.jpeg batch 1 from 156.25
480
0.9375 0.037109375 0.060546875
condcords25149_3d.txt dataimg25149.jpeg batch 1 from 156.25
480
0.9375 0.0625 0.046875
condcords14449_3d.txt dataimg14449.jpeg batch 1 from 156.25
480
0.9375 0.04296875 0.048828125
condcords16212_3d.txt dataimg16212.jpeg batch 1 from 156.25
480
0.9375 0.046875 0.052734375
condcords24625_3d.txt dataimg24625.jpeg batch 1 from 156.25
480
0.9375 0.03515625 0.021484375
condcords16550_3d.txt dataimg16550.jpeg batch 1 from 156.25
480
0.9375 0.05078125 0.0078125
condcords14836_3d.txt dataimg14836.jpeg batch 1 from 156.25
480
0.9375 0.037109375 0.037109375
condcords19264_3d.txt dataimg19264.jpeg batch 1 from 156.25
480
0.9375 0.03515625 0.0546875
condcords00277_3d.txt dataimg00277.jpeg batch 1 from 156.25
480
0.9375 0.0390625 0.013671875
condcords01729_3d.txt dataimg01729.jpeg batch 1 from 156.25
480
0.9375 0.0390625 0.0078125
condcords01935_3d.txt dataimg01935.jpeg batch 1 from 156.25
480
0.9375 0.03

In [3]:
train[0][0].shape


torch.Size([32, 3, 480, 480])

In [4]:
mouth_pointlist = [44, 7, 33, 14, 2, 31, 49, 15, 42, 32, 9, 51, 38, 61,
    18, 23, 12, 47, 67, 1, 2]
mouth_boundaries = [7, 14, 15, 67]
eye_L_pointlist = [62, 65, 0, 13, 34, 64]
eye_R_pointlist = [16, 36, 54, 55, 53, 63]
nose_pointlist = [5, 29, 51, 30, 4, 57]


eyes_corners = [53,36,62,13]

In [5]:
def coords_to_img(coordsbatch, idlist_list, imgsize, blobsize, bglevel):
    imglist = []
    for coords in coordsbatch:
        true_cord_list = []
        for idlist in idlist_list:
            midx = 0.0
            midy = 0.0
            for id in idlist:
                midx += coords[id][0]
                midy += coords[id][1]
            midx /= len(idlist)
            midy /= len(idlist)
            img = np.zeros((imgsize, imgsize), dtype=np.float32)
            x = int(midx * imgsize)
            y = int(midy * imgsize)
            true_cord_list.append(np.array([x, y]))

        for cord in true_cord_list:
            img[cord[1], cord[0]] = 15 * blobsize
        
        img = cv2.GaussianBlur(img, (351, 351), sigmaX=blobsize, borderType=cv2.BORDER_REPLICATE)
        
        for cord in true_cord_list:
            cv2.circle(img, (cord[0], cord[1]), 1, 6, -1)
        
    # img[y, x] = 15000
    # img = cv2.GaussianBlur(img, (351, 351), sigmaX=50, borderType=cv2.BORDER_REPLICATE)
    # cv2.circle(img, (x, y), 0, 6, -1)
    # img = cv2.GaussianBlur(img, (11, 11), sigmaX=1)

        img = cv2.GaussianBlur(img, (11, 11), sigmaX=2)
        img += bglevel
        imglist.append(torch.from_numpy(img).unsqueeze(0))
    return torch.stack(imglist)

In [18]:
def get_mean_coords(landmarks_list, tensor):
    x = torch.zeros(tensor.shape[0]).to(device)
    y = torch.zeros(tensor.shape[0]).to(device)
    for id in landmarks_list:
        x += tensor[:,id,0]
        y += tensor[:,id,1]
    x /= len(landmarks_list)
    y /= len(landmarks_list)
    return x, y

def lstolsofls(listoflist):
    newlist = []
    for listt in listoflist:
        newlist.append([listt])
    return newlist


def crop_center(imgtensor, pointlist, size, coordstens, cordx = None, cordy = None, shift = 0):
    x = 0
    y = 0
    if cordx == None or cordy == None:
        x = torch.zeros(coordstens.shape[0])
        y = torch.zeros(coordstens.shape[0])
        for id in pointlist:
            x += coordstens[:,id,0]
            y += coordstens[:,id,1]
        x /= len(pointlist)
        y /= len(pointlist)
        x *= imgtensor.shape[3]
        y *= imgtensor.shape[2]
        rand = np.random.randint(-shift, shift)
        x += rand
        rand = np.random.randint(-shift, shift)
        y += rand
    else:
        x = cordx * torch.ones(coordstens.shape[0])
        y = cordy * torch.ones(coordstens.shape[0])
    
    left = (x.int() - size)
    top = (y.int() - size)
    right = (x.int() + size)
    bottom = (y.int() + size)
    
    if (torch.min(left) < 0 or torch.min(top) < 0 or 
        torch.max(right) > imgtensor.shape[3] or 
        torch.max(bottom) > imgtensor.shape[2]):
        return None
    
    tenslist = []
    id = 0
    for left_el, top_el, right_el, bottom_el in zip(left, top, right, bottom):
        tenslist.append(imgtensor[id, :, top_el:bottom_el, left_el:right_el])
        id += 1
    return torch.stack(tenslist)

In [19]:
class WMSELoss(nn.Module):
    def __init__(self,  device, l2_lambda = 0.001):
        super().__init__()
        self.device = device
        self.l2_lambda = l2_lambda
        
    def forward(self, x, y,weightmap, parameters = None):
        ls = ((x-y) * weightmap)**2
        sm = ls.sum() / (x.shape[0] * x.shape[1])
        if parameters is None:
            return sm
        
        pk = 0.0
        smp = 0.0
        if parameters is not None:
            for param in parameters:
                smp += (param**2).mean()
                pk += 1

        return sm + (smp / float(pk)) * self.l2_lambda

In [102]:
class CnnDetector(nn.Module):
    def __init__(self, device):
        super(CnnDetector, self).__init__()
        self.adpool = nn.AdaptiveAvgPool2d((128, 128)).to(device)
        self.glpool = nn.AdaptiveAvgPool2d((1, 1)).to(device)
        self.pool = nn.MaxPool2d(2, 2).to(device)
        self.conv1 = nn.Conv2d(3, 5, 3, padding = 1).to(device) 
        self.conv2 = nn.Conv2d(5, 5, 3, padding = 1).to(device)
        self.conv3 = nn.Conv2d(5, 5, 3, padding = 1).to(device)
        self.conv4 = nn.Conv2d(5, 5, 3, padding = 1).to(device)
        self.conv5 = nn.Conv2d(5, 5, 3, padding = 1).to(device)
        self.conv6 = nn.Conv2d(5, 5, 3, padding = 1).to(device)
        self.fc1 = nn.Linear(16 * 16 * 5, 64).to(device)
        self.fc2 = nn.Linear(64, 64).to(device)
        self.fc3 = nn.Linear(64, 64).to(device)
        self.fc4 = nn.Linear(64, 64).to(device)
        self.fc5 = nn.Linear(64, 8).to(device)
        self.act = nn.ReLU().to(device)
        self.sigm = nn.Sigmoid().to(device)

    def forward(self, x, fully_connected = False):
        # Input: [batch, 3, H, W]
        x = self.adpool(x) # 64    
        x1 = x.clone()
        x = self.pool(self.act(self.conv1(x)))
        x = self.act(self.conv2(x)) # 32 
        x += self.pool(x1).repeat(1, 2, 1, 1)[:, :5, :, :] * 0.1
        x1 = x.clone()
        x = self.pool(self.act(self.conv3(x)))
        x = self.act(self.conv4(x)) # 16
        x += self.pool(x1) * 0.1
        x1 = x.clone()
        x = self.act(self.conv5(x))
        x = self.act(self.conv6(x)) # 16
        x += x1 * 0.1
        x = x.view(-1, 16 * 16 * 5)
        x = self.act(self.fc1(x))
        x1 = x.clone()
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x += x1 * 0.1
        x = self.act(self.fc3(x))
        x = self.act(self.fc4(x))
        x += x1 * 0.1
        x = self.fc5(x)
        return x.view(-1, 8)

In [111]:
det_eye = CnnDetector(device).to(device)

det_mouth = CnnDetector(device).to(device)

det_nose = CnnDetector(device).to(device)

#criterion = WMSELoss(device)

criterion = nn.BCELoss().to(device)

learning_rate = 0.001
optimizer_eye = torch.optim.Adam(det_eye.parameters(), lr=learning_rate)
optimizer_mouth = torch.optim.Adam(det_mouth.parameters(), lr=learning_rate)
optimizer_nose = torch.optim.Adam(det_nose.parameters(), lr=learning_rate)

In [113]:


# learning loop
epoch_loss = 0
for epoch in range(20):
    epoch_loss = 0
    step_loss = 0
    step_count = 0
    random.shuffle(train)
    for batch_idx, (inputs, answers) in enumerate(train):
        needshow = torch.tensor(False).to(device)
        if(batch_idx % 3500 == 6):
           needshow = True

        winsize = 63
        sh = 2
        lefteye_img_true = crop_center(inputs, eye_L_pointlist, winsize, answers, shift = sh).to(device)
        righteye_img_true = crop_center(inputs, eye_R_pointlist, winsize, answers, shift = sh).to(device)

        randimglist = []
        for i in range(2):
            yrand = np.random.randint(winsize + 1, inputs.shape[2] - winsize)
            xrand = np.random.randint(winsize + 1, inputs.shape[3] - winsize)
            rand_img = crop_center(inputs, None, winsize, answers, xrand, yrand)
            randimglist.append(rand_img)
        
        rand_img_a = randimglist[0].to(device)
        rand_img_b = randimglist[1].to(device)

        batchsize = inputs.shape[0]

        inp = torch.cat((lefteye_img_true, righteye_img_true, rand_img_a, rand_img_b), 0)
        
        groundtruth = torch.cat((torch.ones(batchsize, 1), torch.ones(batchsize, 1),
                                torch.zeros(batchsize, 1), torch.zeros(batchsize, 1)
                        ), 0).to(device)

        outputs = det_eye(inp)        
        
        # print(groundtruth[2*batchsize-2:2*batchsize+2])
        print(outputs[2*batchsize-2:2*batchsize+2])
        # loss = (outputs * groundtruth).mean() + 1
        loss = criterion(outputs, groundtruth)

        optimizer_eye.zero_grad()
        loss.backward()
        optimizer_eye.step()

        step_loss += loss
        step_count += 1
        if(step_count == 30):
            
            print(f'Batch {batch_idx}, Loss: {step_loss.item() / step_count:.5f}')
            step_count = 0
            step_loss = 0
        epoch_loss += loss.item() 
    
    learning_rate /= 5.0
    optimizer_eye = torch.optim.Adam(det_eye.parameters(), lr=learning_rate)
optimizer_mouth = torch.optim.Adam(det_mouth.parameters(), lr=learning_rate)
optimizer_nose = torch.optim.Adam(det_nose.parameters(), lr=learning_rate)
    # print(f'shape {inputs.shape}, Epoch {epoch + 1}, Loss: {epoch_loss/len(train):.5f}')


tensor([[0.9960],
        [0.9770],
        [0.2792],
        [0.0013]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.9988],
        [0.9756],
        [0.1428],
        [0.1455]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.9963],
        [0.9805],
        [0.3531],
        [0.0630]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.8056],
        [0.9604],
        [0.0154],
        [0.3784]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.5301],
        [0.9965],
        [0.6580],
        [0.0199]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.9306],
        [0.9800],
        [0.0037],
        [0.0140]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.9792],
        [0.5347],
        [0.0042],
        [0.2444]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.9913],
        [0.9915],
        [0.0016],
        [0.0262]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[0.8836],
        [0.9145],
        [0.0235],
        [0.0261]],

In [ ]:
# testloop
criterion = SelectiveRMSELoss(eye_L_pointlist + eye_R_pointlist, False, device)
epoch_loss = 0
test = data_loader.load("I:/NSU/CV/tests/torch/data/test/coords",
                        "I:/NSU/CV/tests/torch/data/test/images",  
                        firstn = 2000, batchSize = 16, shuffle = True, displace = True, 
                        size = 400, show = False)
random.shuffle(test)
                        


In [49]:
model

CnnDetector(
  (adpool): AdaptiveAvgPool2d(output_size=(128, 128))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv11): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv21): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv31): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv41): Conv2d(7, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv22): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv42): Conv2d(7, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv13): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv23): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv33): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv43): 

In [ ]:
with torch.no_grad():
    for batch_idx, (inputs, answers) in enumerate(test):
        inputs = inputs.to(device)
        answers = answers.to(device)
        answers = scaler.transform(answers)
        outputs = model(inputs)
        outputs = scaler.inverse_transform(outputs)
        loss = criterion(outputs, answers)
        epoch_loss += loss.item()

    print(f'Test Loss: {epoch_loss/len(test):.4f}')


In [ ]:
random.shuffle(test)
with torch.no_grad():
    inputs, answers = test[1]
    inputs = inputs.to(device)
    answers = answers.to(device)
    outputs = model(inputs)
    outputs = scaler.inverse_transform(outputs)
    # x_l, y_l = get_mean_coords(eye_L_pointlist, outputs)
    # x_r, y_r = get_mean_coords(eye_R_pointlist, outputs)
    # print(x_l[0], y_l[0], x_r[0], y_r[0])
    # outputs[0][1][0] = x_l[0]
    # outputs[0][1][1] = y_l[0]
    # outputs[0][2][0] = x_r[0]
    # outputs[0][2][1] = y_r[0]
    print(outputs.shape, answers.shape)
    img = modelutils.show_tensor(inputs[0], outputs[0])
    img.show()
    imgdlib = modelutils.show_tensor(inputs[0], answers[0])
    imgdlib.show()



In [114]:


#Save the model after training
torch.save(det_eye.state_dict(), "eyes_window_detector.pth")


In [ ]:
class mouthBoundDetector(nn.Module):
    def __init__(self, device):
        super(mouthBoundDetector, self).__init__()
        self.last_detector_size = 128 
        self.adpool = nn.AdaptiveAvgPool2d((128, 128)).to(device) 
        self.conv1 = nn.Conv2d(3, 6, 3, padding = 1).to(device)
        self.pool = nn.MaxPool2d(2, 2).to(device)
        self.conv2 = nn.Conv2d(6, 9, 3, padding = 1).to(device)
        self.conv3 = nn.Conv2d(9, 20, 3, padding = 1).to(device)
        self.conv4 = nn.Conv2d(20, self.last_detector_size, 3, padding = 1).to(device)
        fcsize = 256
        self.fc1 = nn.Linear(self.last_detector_size*16*16, fcsize).to(device)
        self.fc_list = []
        for i in range(3):
            self.fc_list.append(nn.Linear(fcsize, fcsize).to(device))
        self.prelast = nn.Linear(fcsize, fcsize).to(device)
        self.fc_last = nn.Linear(fcsize, 3 * 68).to(device)
        self.act = nn.ReLU().to(device)
        self.sigm = nn.Sigmoid().to(device)
    
    def forward(self, x):
        # Input: [batch, 3, H, W]
        x = self.adpool(x)
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = self.act(self.conv3(x))  
        x = self.pool(self.act(self.conv4(x)))
        x = x.view(-1, self.last_detector_size*16*16)       
        x = self.act(self.fc1(x))
        for i in range(len(self.fc_list)):
            x = self.act(self.fc_list[i](x))
        x = self.act(self.prelast(x))
        x = self.fc_last(x)
        x = x.view(-1, 68, 3)
        return x


In [ ]:
with torch.no_grad():
    inputs, answers = test[1]
    inputs = inputs.to(device)
    answers = answers.to(device)
    outputs = model(inputs)
    outputs = scaler.inverse_transform(outputs)
    print(outputs.shape, answers.shape)
    img = show_tensor(inputs[0], outputs[0])
    img.show()
    imgdlib = show_tensor(inputs[0], answers[0])
    imgdlib.show()